# PCY ALGORITHM

In [2]:
import itertools
import requests
import findspark
findspark.init()
from pyspark.conf import SparkConf
from pyspark import SparkContext
from nltk.corpus import stopwords

Load the stop words and add rt(retweet) as stopword

In [3]:
stopWords = set(stopwords.words("english"))
stopWords.add('rt')

In [4]:
conf = SparkConf().setAppName("tweet").setMaster("local")

In [5]:
sc = SparkContext(conf=conf)

Define function toSetOfWords that take the words from the tweet and remove all the stopwords

In [6]:
def toSetOfWords(tweet):
    return (set(tweet.lower().replace('\\n','').split())-stopWords)

Take every tweet and split it in tuples counting by one

In [7]:
def toSet(tweet):    
    wordCount = set()
    for word in tweet:
        wordCount.add((word, 1))
    return wordCount

Generate pairs using frequentItems

In [25]:
def getSubset(t, size, frequentItems):
    subSetsFromBasket = set()
    combinations = itertools.combinations(t, size)
    for subset in combinations:
        #print("{} {} - {} {}".format(subset[0], subset[0] in frequentItems, subset[1], subset[1] in frequentItems))
        if subset[0] in frequentItems and subset[1] in frequentItems:
            #print("SIIIII")
            subSetsFromBasket.add((subset, 1))
    return subSetsFromBasket

In [60]:
def get_solr_result_as_string(query):
    result = requests.get("http://localhost:8983/solr/twitter/select?q=*:*")
    return result.content

In [9]:
data = sc.textFile("file:///home/han/tweettextall.txt")

In [11]:
tweetSetOfWords = data.map(toSetOfWords)
tweetWords = tweetSetOfWords.flatMap(toSet)

In [12]:
tweetSetOfWords.take(2)

[{'2019',
  '@gahbrielle_:',
  'almost',
  'it’s',
  'leaving',
  'marvel',
  'movie?',
  'right',
  'still',
  'theater',
  'y’all'},
 {'5',
  'bad',
  'christmas',
  'chunk',
  'die',
  'hacky',
  'hard',
  "i'm",
  'louis',
  'minute',
  'movie.',
  'really',
  'set',
  'surprised',
  'whether'}]

In [13]:
tweetWords.take(20)

[('right', 1),
 ('y’all', 1),
 ('2019', 1),
 ('marvel', 1),
 ('almost', 1),
 ('@gahbrielle_:', 1),
 ('movie?', 1),
 ('theater', 1),
 ('leaving', 1),
 ('it’s', 1),
 ('still', 1),
 ('really', 1),
 ('set', 1),
 ('bad', 1),
 ('whether', 1),
 ('hacky', 1),
 ('movie.', 1),
 ('louis', 1),
 ('5', 1),
 ('die', 1)]

In [16]:
transactionCount = data.count()
print(transactionCount)
minSupport = transactionCount * 0.1
print(minSupport)

589156
58915.600000000006


In [15]:
reducedWordsRdd = tweetWords.reduceByKey(lambda a, b: a + b)

In [19]:
singletonSupport = dict()
for s in reducedWordsRdd.collect():
    singletonSupport[s[0]]= s[1]

In [20]:
frequentItems = reducedWordsRdd.filter(lambda w: w[1] >= minSupport).map(lambda w: w[0]).collect()

In [21]:
frequentItems = set(frequentItems)
frequentItems

{'movie'}

In [23]:
itemPairs = tweetSetOfWords.flatMap(lambda t: getSubset(t, 2, frequentItems))

In [24]:
itemPairs.take(5)

[]

In [108]:
itemPairsCount = itemPairs.reduceByKey(lambda a, b: a + b)

In [109]:
itemPairsCount.take(5)

[(('movie', 'literally'), 72),
 (('“us”', 'proud'), 61),
 (('h…', 'movie??'), 61),
 (('family', 'version'), 69),
 (('version', 'soooo'), 61)]

In [110]:
frequentItemPairs = itemPairsCount.filter(lambda w: w[1] >= minSupport)

In [111]:
frequentItemPairs.take(5)

[(('movie', 'literally'), 72),
 (('“us”', 'proud'), 61),
 (('h…', 'movie??'), 61),
 (('family', 'version'), 69),
 (('version', 'soooo'), 61)]

In [112]:
def setSupportAndConfidence(x):
    s = x[1]/transactionCount
    return (x[0], s, s/singletonSupport[x[0][0]], s/singletonSupport[x[0][1]])

In [113]:
supports = frequentItemPairs.map(setSupportAndConfidence)

In [115]:
supports.take(5)

[(('movie', 'literally'), 0.144, 0.0004542586750788643, 0.0019459459459459458),
 (('“us”', 'proud'), 0.122, 0.002, 0.0017428571428571428),
 (('h…', 'movie??'), 0.122, 0.0019677419354838708, 0.002),
 (('family', 'version'), 0.138, 0.0019166666666666668, 0.002),
 (('version', 'soooo'), 0.122, 0.0017681159420289854, 0.002)]